In [ ]:
# Run in python console
import nltk; nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/apex-codes/entity_sum

/content/drive/My Drive/Colab Notebooks/apex-codes/entity_sum


In [ ]:
import os
import pandas as pd
import pickle as pk
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Compute Novelty

In [ ]:
!python3 -m spacy download en

     |████████████████████████████████| 12.0 MB 4.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import sys
import re
import os
import csv
import string
import json
import spacy
import nltk
import gensim
import collections
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')
from gensim.models import Phrases
from gensim.utils import simple_preprocess

from collections import defaultdict

#%matplotlib inline

stop_words = set(stopwords.words("english")) 
stopwords = str(stop_words)
stop = open('terrier-stop.txt')
stopString = stop.read()
common_terms = stopString.split()

## Utility functions

In [ ]:
# Utility functions

def preprocess_data(data):
  data = data
  # Remove Emails
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
  # Remove new line characters
  data = [re.sub('\s+', ' ', sent) for sent in data]
  # Remove distracting single quotes
  data = [re.sub("\'", "", sent) for sent in data]

  return data

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

######################################################################################

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    #!python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#####################################################################################

def nlp_process(data_words, bigram_mod, trigram_mod):
  # Remove Stop Words
  data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words_nostops, bigram_mod)

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams)

  return data_lemmatized

### Unigrams generator function

In [ ]:
# This function returns unigrams for 1) abstract or 2) the collection of topic-aware citation contexts---so it must be passed the raw abstrac
# or the raw citation contexts---this one generates top 50 unigrams
def _generate_unigrams(text):
  unigrams_list = []

  words = nltk.word_tokenize(text)
  fileList = [word.lower() for word in words if word.isalpha()]
  freeList = [t for t in fileList if t not in common_terms and t not in stop_words and t not in string.punctuation]
  freeString = [t for t in fileList if t not in common_terms and t not in stop_words and t not in string.punctuation]
  a = ' '.join(freeString)       #conversion into a string
  wordcount = {}
  for word in freeList:      
      if word not in stopwords:
          if word not in wordcount:
              wordcount[word] = 1
          else:
              wordcount[word] += 1
  word_counter = collections.Counter(wordcount)
  # Just grab the 50 most frequently occuring unigrams---number can change accordingly
  for word, count in word_counter.most_common(50):
      unigrams_list.append(word)

  return unigrams_list


### Bigrams and trigrams generator function

In [ ]:
def _generate_bigrams_trigrams(text):
  #bigrams_list = []
  # tokenize the text into individual sentences that are stored in a list
  filtered_data = sent_tokenize(text)

  data = preprocess_data(filtered_data)

  #######################################################################################
  data_words = list(sent_to_words(data))


  # Build the bigram and trigram models
  bigram = gensim.models.Phrases(data_words, min_count=1, threshold=1) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], min_count=1, threshold=1)  

  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  data_lemmatized = nlp_process(data_words, bigram_mod, trigram_mod)

  sentence_stream = data_lemmatized

  bigrams_lemmatized = []
  trigrams_lemmatized = []

  bigram = Phrases(sentence_stream, min_count=3, delimiter=b'_')
  trigram  = Phrases(bigram[sentence_stream], min_count=3, delimiter=b'_')
  for sent in sentence_stream:
      bigrams_ = [b for b in bigram[sent] if b.count('_') == 1]
      trigrams_ = [t for t in trigram[bigram[sent]] 
                                            if t.count('_')==2]
      bigrams_lemmatized.append(bigrams_)
      trigrams_lemmatized.append(trigrams_)
  
  ngrams_lemmatized =  trigrams_lemmatized + bigrams_lemmatized

  return bigrams_lemmatized, trigrams_lemmatized
  #return ngrams_lemmatized

# Compute N-gram Novelty with reference to source articles

---



In [ ]:
def _compute_novelty_I(gen_summary, human_summary):
  # generate unigrams for the summary and the abstract
  gen_summary_unigrams = _generate_unigrams(gen_summary)
  human_summary_unigrams = _generate_unigrams(human_summary)

  # generate bigrams and trigrams for the summary and the abstract
  gen_summary_bi_trigrams, gen_summary_tri_trigrams = _generate_bigrams_trigrams(gen_summary)
  human_summary_bi_trigrams, human_summary_tri_trigrams = _generate_bigrams_trigrams(human_summary)

  # Total n-grams for the summary and the abstract
  gen_summary_ngrams = gen_summary_unigrams + gen_summary_bi_trigrams + gen_summary_tri_trigrams
  human_summary_ngrams = human_summary_unigrams + human_summary_bi_trigrams + human_summary_tri_trigrams


  diff_list = [ngram for ngram in gen_summary_ngrams if ngram not in human_summary_ngrams]   # unigram that is in a summary but not in the abstract

  novelty_score = float(len(diff_list)) / float(len(gen_summary_ngrams))
  novelty_score = novelty_score * 100

  return novelty_score

  #print(f"{RESULTS_PATH} Novelty Score wrt abstracts: %.2f" % novelty_score)


## DON'T DO Novelty wrt human summary - Evaluation against ground truth summaries (Yet, don't do this)

## Evaluation against the source articles themselves

In [ ]:
def main():
  lst_modelName = ["T5", "BART", "Pegasus"]
  lst_model_type = ["vanilla", "w_named_entities"]

  dict_novelty_scores = defaultdict(list)

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      SUMMARY_PATH = f"pubmed-FINAL-SUMMARIES/{modelName}"
      input_filename_path = f"{SUMMARY_PATH}/pubmed-summaries-{model_type}.jsonl"

      output_results_path = f"FINAL_RESULTS/NOVELTY"
      os.makedirs(output_results_path, exist_ok=True)
      output_file_name = "n_gram_novelty_scores.csv"

      novelty_sum = 0.0   # initialize all cumulative scores to zero
      total_no_summaries = 0
      with open(input_filename_path) as fp:
        for iter, line in enumerate(fp):
          if iter % 2000 == 0:
            print("Iteration: ", iter)
          dict_data = json.loads(line)

          generated_summary = dict_data["abstractive_summary"]    # generated summary
          #ground_truth_summary = dict_data["article_abstract"]   # human-like summary
          article_text = dict_data["article_text"]   # source input article----to do evaluation wrt the input article

          # call to the ROUGE computing method
          try:
            novelty_score = _compute_novelty_I(generated_summary, article_text)

            novelty_sum += novelty_score
            
            total_no_summaries += 1

          except:
            continue

      fp.close()

      avg_novelty = novelty_sum/float(total_no_summaries)

      dict_novelty_scores["training-config"].append(f"{modelName}-{model_type}")
      dict_novelty_scores["n-gram-novelty"].append(avg_novelty)

  df_novelty_scores = pd.DataFrame(dict_novelty_scores)

  print(df_novelty_scores)

  df_novelty_scores.to_excel("n_gram_novelty_scores.xlsx")  

In [ ]:
if __name__ == "__main__":
  main()

Iteration:  0
Iteration:  2000
Iteration:  4000
Iteration:  0
Iteration:  2000
Iteration:  4000
Iteration:  0
Iteration:  2000
Iteration:  4000
Iteration:  0
Iteration:  2000
Iteration:  4000
Iteration:  0
Iteration:  2000
Iteration:  4000
Iteration:  0
Iteration:  2000
Iteration:  4000
            training-config  n-gram-novelty
0                T5-vanilla       52.930007
1       T5-w_named_entities       50.079039
2              BART-vanilla       54.816223
3     BART-w_named_entities       54.959429
4           Pegasus-vanilla       51.260270
5  Pegasus-w_named_entities       52.558129
